In [1]:
from math import ceil
import time

In [2]:
class Benchmark:
    def __init__(self, file_path, benchmark_type):
        self.file_path = file_path
        self.benchmark_type = benchmark_type
        self.subsets, self.universe_size, self.num_subsets = self.read_benchmark()
    
    def read_benchmark(self):
        """Lecture du benchmark"""
        with open(self.file_path, "r") as file:
            lines = file.readlines()
        
        m, n = map(int, lines[0].split())
        cost_lines_to_skip = ceil(n / 12) if self.benchmark_type == "4" else ceil(n / 15)
        subset_start_index = 1 + cost_lines_to_skip
        
        data_lines = lines[subset_start_index:]
        row_to_subsets = {}
        index = 0
        
        for row in range(1, m + 1):
            num_subsets = int(data_lines[index].strip())
            index += 1
            subsets = []
            while len(subsets) < num_subsets:
                subsets.extend(map(int, data_lines[index].split()))
                index += 1
            row_to_subsets[row] = subsets
        
        subset_to_rows = {}
        for row, subsets in row_to_subsets.items():
            for subset in subsets:
                if subset not in subset_to_rows:
                    subset_to_rows[subset] = []
                subset_to_rows[subset].append(row)
        return subset_to_rows, m, n

In [3]:
class DFSSolver:
    def __init__(self, benchmark, timeout=300):
        self.benchmark = benchmark
        self.k = ceil(benchmark.universe_size * 2 / 3)  # k basé sur la taille de l'univers
        self.timeout = timeout
        self.start_time = None
        self.best_solution = []
        self.best_coverage = 0
        self.nodes_explored = 0
    
    def solve(self):
        """Exécution principale"""
        self.start_time = time.time()
        remaining = set(self.benchmark.subsets.keys())
        self.dfs(remaining, [], set())
        
        return {
            "k": self.k,
            "selected_subsets": self.best_solution[:self.k],  # Garantit ≤k subsets
            "coverage": self.best_coverage,
            "coverage_ratio": self.best_coverage / self.benchmark.universe_size,
            "nodes_explored": self.nodes_explored,
            "time_taken": time.time() - self.start_time
        }
    
    def dfs(self, remaining, selected, covered):
        """DFS avec élagage intelligent"""
        if self.check_timeout() or len(selected) > self.k:
            return
        
        self.nodes_explored += 1
        
        current_cov = len(covered)
        if current_cov > self.best_coverage:
            self.best_coverage = current_cov
            self.best_solution = selected.copy()
            print(f"Amélioration: {self.best_coverage}/{self.benchmark.universe_size}")
        
        # Élagage si solution complète trouvée
        if self.best_coverage == self.benchmark.universe_size:
            return
        
        # Sélection intelligente des sous-ensembles
        for subset in sorted(remaining, 
                           key=lambda s: len(set(self.benchmark.subsets[s]) - covered),
                           reverse=True):
            if self.check_timeout():
                return
                
            new_covered = covered | set(self.benchmark.subsets[subset])
            self.dfs(remaining - {subset}, selected + [subset], new_covered)
    
    def check_timeout(self):
        elapsed = time.time() - self.start_time
        if elapsed > self.timeout:
            print(f"Timeout après {elapsed:.1f}s")
            return True
        return False


In [4]:

if __name__ == "__main__":
    benchmark = Benchmark("./Benchmark/A/scpa1.txt", "A")
    print(f"Taille univers: {benchmark.universe_size}")
    print(f"k calculé: {ceil(benchmark.universe_size * 2 / 3)}")
    
    solver = DFSSolver(benchmark, timeout=60)
    result = solver.solve()
    
    print("\nRésultats finaux:")
    print(f"Subsets sélectionnés: {len(result['selected_subsets'])}/{result['k']}")
    print(f"Couverture: {result['coverage']}/{benchmark.universe_size}")
    print(f"Ratio: {result['coverage_ratio']:.1%}")
    print(f"Noeuds explorés: {result['nodes_explored']}")
    print(f"Temps: {result['time_taken']:.2f}s")



Taille univers: 300
k calculé: 200
Amélioration: 17/300
Amélioration: 32/300
Amélioration: 46/300
Amélioration: 59/300
Amélioration: 71/300
Amélioration: 82/300
Amélioration: 93/300
Amélioration: 104/300
Amélioration: 115/300
Amélioration: 126/300
Amélioration: 136/300
Amélioration: 146/300
Amélioration: 155/300
Amélioration: 164/300
Amélioration: 173/300
Amélioration: 181/300
Amélioration: 189/300
Amélioration: 196/300
Amélioration: 203/300
Amélioration: 210/300
Amélioration: 217/300
Amélioration: 223/300
Amélioration: 229/300
Amélioration: 235/300
Amélioration: 240/300
Amélioration: 245/300
Amélioration: 250/300
Amélioration: 255/300
Amélioration: 260/300
Amélioration: 264/300
Amélioration: 268/300
Amélioration: 272/300
Amélioration: 276/300
Amélioration: 280/300
Amélioration: 283/300
Amélioration: 286/300
Amélioration: 289/300
Amélioration: 292/300
Amélioration: 294/300
Amélioration: 296/300
Amélioration: 298/300
Amélioration: 300/300

Résultats finaux:
Subsets sélectionnés: 42/200
